Enfoque del dataset de autextication basado en algoritmos clásicos de aprendizaje automático:

In [1]:
### IMPORTS ###
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from keras.utils import to_categorical

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Cargar Dataset

In [2]:
### GOOGLE DRIVE ###
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/TFM/Data/Autextication/"

Mounted at /content/drive


In [3]:
### PARTICIÓN ###
df = pd.read_csv(PATH + "train_en.tsv",  sep='\t', on_bad_lines='skip')
df = df[['text', 'label']]
df = df.rename(columns={"text": "Text", "label": "Label"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=55, stratify=y_train)

X_train, X_aux, y_train, y_aux = train_test_split(X_train, y_train, test_size=0.4, random_state=55, stratify=y_train)
X_test, X_aux, y_test, y_aux = train_test_split(X_test, y_test, test_size=0.4, random_state=55, stratify=y_test)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Evaluación:', len(X_test))

Tamaño conjunto de Entrenamiento: 16245
Tamaño conjunto de Evaluación: 4061


# Limpieza y Representación de Textos

In [4]:
### LIMPIEZA DE TEXTOS ###
stopwords_es = stopwords.words("english")
def clean_text(text):
    # transformar a minúscula
    text=str(text).lower()
    # tokenizar
    tokens=word_tokenize(text)
    # borrar stopwords
    tokens = [word for word in tokens if word not in stopwords_es]
    # usar los stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # eliminamos las palabras con menos de 2 caráceres
    # ignoramos cualquier palabra que contenga un digito o un símbolo especial
    min_length = 2
    p = re.compile('^[a-zA-Z]+$');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)

    return filtered_tokens

# Data Augmentation

No es necesario hacer Data Augmentation porque el corpus es lo suficientemente grande

# Label Encoding

In [5]:
y_train = y_train.replace(["human", "generated"], [0, 1])

y_test = y_test.replace(["human", "generated"], [0, 1])

# Bolsa de Palabras

In [6]:
### BOLSA DE PALABRAS ###
X_train = X_train.tolist()
X_test = X_test.tolist()

# entrenamos un modelo de bolsa de palabras
bow = CountVectorizer(analyzer=clean_text).fit(X_train)
# transformamos el conjunto de entrenamiento a bolsa de palabras
X_train_bow = bow.transform(X_train)
# transformamos el conjunto de evaluación a bolsa de palabras
X_test_bow = bow.transform(X_test)

print("Tamaño del vocabulario: ", len(bow.vocabulary_))

Tamaño del vocabulario:  19293


# TF-IDF

In [7]:
### TF-IDF ###
# entrenamos un modelo tf-idf
tfidf_transformer = TfidfTransformer().fit(X_train_bow)
# transformamos el conjunto de entrenamiento
X_train_tfidf = tfidf_transformer.transform(X_train_bow)
# transformamos el conjunto de entrenamiento
X_test_tfidf = tfidf_transformer.transform(X_test_bow)

# Clasificación Clásica
Se crea un pipeline que ejecuta una secuencia de procesos:


1.   La representación de los textos en bolsa de palabras (CountVectorizer), que recibe como entrada los textos, y se les aplica dentro del CountVectorizer la función clean_text para limpiarlos y reducir el ruido.
2.   La representación en tf-idf (TfidfTransformer), recibe como entrada la salida del proceso 1, y produce los vectores tf-idf.
3. El clasificador SVC, Logistic Regression o Random Forest Clasiffier.

In [8]:
### PIPELINE SVM ###
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('svm', SVC()),
])

# Parámetros para el algoritmo SVM
grid_params_svm = [{'svm__kernel': ['linear'],
                    'svm__C': [0.1, 1], # [0.1, 1, 10, 100, 1000]
                    'svm__gamma':  [1, 0.1] # [1, 0.1, 0.01, 0.001, 0.0001]
                    }]
gs = GridSearchCV(pipeline, param_grid=grid_params_svm,
                  scoring='accuracy', cv=2, verbose = 1)

# entrenamos el grid
gs.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs.best_params_)
print('Mejor accuracy: %.3f' % gs.best_score_)
print(gs.best_estimator_)

best_svm = gs.best_estimator_
predictions = best_svm.predict(X_test)
print(classification_report(y_test, predictions))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Los mejores parámetros son : {'svm__C': 1, 'svm__gamma': 1, 'svm__kernel': 'linear'}
Mejor accuracy: 0.713
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f843b9f5240>)),
                ('tf', TfidfTransformer()),
                ('svm', SVC(C=1, gamma=1, kernel='linear'))])
              precision    recall  f1-score   support

           0       0.72      0.72      0.72      2045
           1       0.72      0.72      0.72      2016

    accuracy                           0.72      4061
   macro avg       0.72      0.72      0.72      4061
weighted avg       0.72      0.72      0.72      4061



In [9]:
### PIPELINE LOGISTIC REGRESSION ###
pipeline2 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('lr', LogisticRegression(random_state=0)),
])

# Parámetros para el algoritmo Logistic Regression
grid_params_lr = [{'lr__penalty': ['l1', 'l2'],
                    'lr__C': [1.0, 0.5],
                    'lr__solver':  ['liblinear']
                    }]
gs2 = GridSearchCV(pipeline2, param_grid=grid_params_lr,
                  scoring='accuracy', cv=3, verbose = 1)

# entrenamos el grid
gs2.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs2.best_params_)
print('Mejor accuracy: %.3f' % gs2.best_score_)
print(gs2.best_estimator_)

best_svm = gs2.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Los mejores parámetros son : {'lr__C': 1.0, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}
Mejor accuracy: 0.727
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f843b9f5240>)),
                ('tf', TfidfTransformer()),
                ('lr', LogisticRegression(random_state=0, solver='liblinear'))])
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      2045
           1       0.72      0.74      0.73      2016

    accuracy                           0.73      4061
   macro avg       0.73      0.73      0.73      4061
weighted avg       0.73      0.73      0.73      4061



In [10]:
### PIPELINE RANDOM FOREST ###
pipeline3 = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_text)),
    ('tf', TfidfTransformer()),
    ('rfc', RandomForestClassifier(random_state=0)),
])

# Parámetros para el algoritmo Random Forest
grid_params_rfc = [{'rfc__criterion': ['gini', 'entropy'],
                    'rfc__max_depth': [9, 10],
                    'rfc__min_samples_split':  [10]
                    }]
gs3 = GridSearchCV(pipeline3, param_grid=grid_params_rfc,
                  scoring='accuracy', cv=3, verbose = 1)

# entrenamos el grid
gs3.fit(X_train, y_train)
print('Los mejores parámetros son : %s' % gs3.best_params_)
print('Mejor accuracy: %.3f' % gs3.best_score_)
print(gs3.best_estimator_)

best_svm = gs3.best_estimator_
predictions = best_svm.predict(X_test)
print( classification_report(y_test, predictions))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Los mejores parámetros son : {'rfc__criterion': 'gini', 'rfc__max_depth': 10, 'rfc__min_samples_split': 10}
Mejor accuracy: 0.639
Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function clean_text at 0x7f843b9f5240>)),
                ('tf', TfidfTransformer()),
                ('rfc',
                 RandomForestClassifier(max_depth=10, min_samples_split=10,
                                        random_state=0))])
              precision    recall  f1-score   support

           0       0.89      0.32      0.47      2045
           1       0.58      0.96      0.72      2016

    accuracy                           0.64      4061
   macro avg       0.73      0.64      0.60      4061
weighted avg       0.73      0.64      0.59      4061

